In [46]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

load_dotenv()



True

In [47]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [48]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

In [49]:
llm = AzureChatOpenAI(
    openai_api_version=os.getenv('OPENAI_API_VERSION'),
    azure_deployment=os.getenv('AZURE_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT'),
    temperature=0, 
    max_tokens=500
)
server = 'ews.database.windows.net'
database = 'ews_assets'
username = 'ews_admin'
password = 'Test1234!' 
driver = '{ODBC Driver 18 for SQL Server}' 

connection_string = (
    "mssql+pyodbc://{username}:{password}@{server}/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
).format(
    username=username,
    password=password,
    server='ews.database.windows.net',
    database = 'ews_assets'
)
db = SQLDatabase.from_uri(connection_string)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [53]:
QUESTION = """In the I-OBS table how many records there are with Wx_Model_Hour = 0, Take in consideration only the the records with the following EWS_ID 5136,
5134, 5133, 5205, 5142, 5143, 5141, 5289, 5290, 5291, 5292, 5118, 5121, 5125, 5126, 5127, 5128, 5184, 5525
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True,
    handle_parsing_errors=True
)

In [54]:
agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Clarity_Unit, ERCOT_Region, ERCOT_WeatherRegion, EWS, EWS_Location, EWS_USGS_Summary, GenerationData, I-OBS, IIR_Location, IIR_Operator, IIR_Owner, IIR_Parent, IIR_Plant, IIR_Turbine, IIR_Unit, IIR_WindPowerCurves, USGS, sysdiagramsThe I-OBS table is available, so I should check its schema to understand which columns are available and how to properly construct my query.
Action: sql_db_schema
Action Input: I-OBS
CREATE TABLE [I-OBS] (
	[ID] BIGINT NOT NULL IDENTITY(1,1), 
	[EWS_ID] INTEGER NOT NULL, 
	[Wx_Source_Alias] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Wx_Model_Run_DateTime_UTC] DATETIME NOT NULL, 
	[Wx_Model_Name_Run] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Wx_Model_Hour] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Receipt_Time_LocalTime] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Forecast_Valid_DateTime_LocalTime] DATETIME 

{'input': 'In the I-OBS table how many records there are with Wx_Model_Hour = 0, Take in consideration only the the records with the following EWS_ID 5136,\n5134, 5133, 5205, 5142, 5143, 5141, 5289, 5290, 5291, 5292, 5118, 5121, 5125, 5126, 5127, 5128, 5184, 5525\n',
 'output': "There are 861,089 records in the I-OBS table with Wx_Model_Hour = 0 for the specified EWS_IDs.\n\nExplanation:\nI queried the `I-OBS` table to count the number of records where `Wx_Model_Hour` is '0' and `EWS_ID` is within the specified list of IDs. The SQL query used was:\n\n```sql\nSELECT COUNT(*) FROM [I-OBS] WHERE Wx_Model_Hour = '0' AND EWS_ID IN (5136, 5134, 5133, 5205, 5142, 5143, 5141, 5289, 5290, 5291, 5292, 5118, 5121, 5125, 5126, 5127, 5128, 5184, 5525)\n```\n\nThis query returned a count of 861,089, indicating the total number of records that match the criteria."}

In [10]:
QUESTION2 = """In the I-OBS table how many records there are with Wx_Model_Hour = 0, Take in consideration only the the records with the following EWS_ID 5136
5134, 5133, 5205, 5142, 5143, 5141, 5289, 5290, 5291, 5292, 5118, 5121, 5125, 5126, 5127, 5128, 5184, 5525
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [11]:
agent_executor_SQL.invoke(QUESTION2)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Clarity_Unit, ERCOT_Region, ERCOT_WeatherRegion, EWS, EWS_Location, EWS_USGS_Summary, GenerationData, I-OBS, IIR_Location, IIR_Operator, IIR_Owner, IIR_Parent, IIR_Plant, IIR_Turbine, IIR_Unit, IIR_WindPowerCurves, USGS, sysdiagramsThe table "I-OBS" seems to be the relevant table for the query. I should check the schema of the "I-OBS" table to understand its structure and confirm the presence of the columns "Wx_Model_Hour" and "EWS_ID".
Action: sql_db_schema
Action Input: I-OBS
CREATE TABLE [I-OBS] (
	[ID] BIGINT NOT NULL IDENTITY(1,1), 
	[EWS_ID] INTEGER NOT NULL, 
	[Wx_Source_Alias] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Wx_Model_Run_DateTime_UTC] DATETIME NOT NULL, 
	[Wx_Model_Name_Run] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Wx_Model_Hour] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Receipt_Time_LocalTime] NVARCHAR(50) COLLATE SQL_Latin1_Gen

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I don't know`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 